In [12]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [13]:
# get data
data = pd.read_csv('Level_3c_NonPath_SP3_PsPu_PSL_1_2_3_435.csv') 

# split data
X=data.iloc[:,51:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

     Schneider.Xr.A  Schneider.Xr.R  ...  Grantham.Xd.29  Grantham.Xd.30
0          0.110251        0.068907  ...        0.033889        0.032943
1          0.093147        0.067743  ...        0.033243        0.031827
2          0.074997        0.058534  ...        0.032000        0.030636
3          0.154118        0.071351  ...        0.031503        0.032314
4          0.144669        0.070916  ...        0.031943        0.032270
..              ...             ...  ...             ...             ...
430        0.062749        0.038615  ...        0.029650        0.032397
431        0.084790        0.064235  ...        0.032343        0.028946
432        0.163940        0.032788  ...        0.033358        0.028789
433        0.105041        0.010004  ...        0.032696        0.027279
434        0.105041        0.010004  ...        0.032696        0.027279

[435 rows x 100 columns]
0      1
1      1
2      1
3      1
4      1
      ..
430    3
431    3
432    3
433    3
434    3

In [6]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [7]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(100, 3 - 1) = 2 components.
  ChangedBehaviorWarn

In [8]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(3 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 100)            40800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 128)            117248    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 207,651
Trainable params: 207,651
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
21/21 [==============================] - 34s 155ms/step - loss: 0.6836 - accuracy: 0.5126 - val_loss: 0.6241 - val_accuracy: 0.5868
Epoch 2/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.5898 - accuracy: 0.5766 - val_loss: 0.4457 - val_accuracy: 0.5868
Epoch 3/1000
21/21 [==============================] - 0s 19ms/step - loss: 0.4421 - accuracy: 0.6577 - val_loss: 0.3706 - val_accuracy: 0.8084
Epoch 4/1000
21/21 [==============================] - 0s 17ms/step - loss: 0.3723 - accuracy: 0.8090 - val_loss: 0.2514 - val_accuracy: 0.9401
Epoch 5/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.3069 - accuracy: 0.8119 - val_loss: 0.1815 - val_accuracy: 0.9401
Epoch 6/1000
21/21 [==============================] - 0s 18ms/step - loss: 0.2553 - accuracy: 0.8435 - val_loss: 0.1624 - val_accuracy: 0.9401
Epoch 7/1000
21/21 [==============================] - 0s 17ms/step - loss: 0.2389 - accuracy: 0.8518 - val_loss: 0.1449 - val_accuracy: 0.95

In [11]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.952023988005997
test-acc = 0.9461077844311377
train-CFR =               precision    recall  f1-score   support

           0       0.94      0.94      0.94       220
           1       0.95      0.97      0.96       230
           2       0.97      0.95      0.96       217

    accuracy                           0.95       667
   macro avg       0.95      0.95      0.95       667
weighted avg       0.95      0.95      0.95       667

test-CFR =               precision    recall  f1-score   support

           0       0.93      0.93      0.93        58
           1       0.92      0.92      0.92        48
           2       0.98      0.98      0.98        61

    accuracy                           0.95       167
   macro avg       0.94      0.94      0.94       167
weighted avg       0.95      0.95      0.95       167

